<a href="https://colab.research.google.com/github/mdogdope/CSE-212-Final/blob/master/Copy_of_starter_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')

In [ ]:
campaign.head()


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [ ]:
campaign.replace('unknown', pd.NA, inplace=True)
campaign.dropna(inplace=True)

In [ ]:
campaign.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27477 entries, 0 to 37068
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             27477 non-null  int64  
 1   job             27477 non-null  object 
 2   marital         27477 non-null  object 
 3   education       27477 non-null  object 
 4   default         27477 non-null  object 
 5   housing         27477 non-null  object 
 6   loan            27477 non-null  object 
 7   contact         27477 non-null  object 
 8   month           27477 non-null  object 
 9   day_of_week     27477 non-null  object 
 10  campaign        27477 non-null  int64  
 11  pdays           27477 non-null  int64  
 12  previous        27477 non-null  int64  
 13  poutcome        27477 non-null  object 
 14  emp.var.rate    27477 non-null  float64
 15  cons.price.idx  27477 non-null  float64
 16  cons.conf.idx   27477 non-null  float64
 17  euribor3m       27477 non-null 

# Define X and y

In [ ]:
# Define X and y for the train test split (What features do we want to use for X?)
X = campaign[['age','job','marital','education','default','housing','loan','month','day_of_week', 'emp.var.rate', 'cons.price.idx', 'euribor3m', 'nr.employed']]

# Create a new dataframe called y that contians the target we're
# trying to predict. In this case, that is the 'y' column in the dataset
y = campaign['y']

#Unbalance data
from imblearn.over_sampling import RandomOverSampler
ro = RandomOverSampler()
# Oversample, note that we oversample X and y at the same time in order to
# make sure our features and targets stay synched.
X_new, y_new = ro.fit_resample(X, y)
invest = pd.DataFrame(y_new)
invest.value_counts()

y  
no     23979
yes    23979
dtype: int64

# Descision Tree Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Identify categorical columns and separate them for modification
categorical_cols = X.select_dtypes(include=['object']).columns

# Create a column transformer to make the object values One-Hot encoded
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_cols)
    ],
    remainder='passthrough'
)

# Pipeline: Preprocessor (One-Hot) -> Decision Tree Classifier
dModel = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=0, class_weight='balanced'))
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the model
dModel.fit(X_train, y_train)

# Make predictions with probabilities
y_prob = dModel.predict_proba(X_test)[:, 1]

# Adjust the threshold
threshold = 0.3  # Experiment with different values
y_pred_adjusted = (y_prob > threshold).astype(int)

# Map the labels back to original 'yes' and 'no'
label_mapping = {0: 'no', 1: 'yes'}
y_pred_adjusted_labels = np.vectorize(label_mapping.get)(y_pred_adjusted)

# Evaluate the adjusted predictions
accuracy = accuracy_score(y_test, y_pred_adjusted_labels)
print(f"Adjusted Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred_adjusted_labels))

Adjusted Accuracy: 0.81
              precision    recall  f1-score   support

          no       0.90      0.89      0.89      4773
         yes       0.31      0.33      0.32       723

    accuracy                           0.81      5496
   macro avg       0.60      0.61      0.61      5496
weighted avg       0.82      0.81      0.82      5496



# Random Forest Model

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Assume X and y are defined somewhere before this code

# Identify categorical columns and separate them for modification
categorical_cols = X.select_dtypes(include=['object']).columns

# Create a column transformer to make the object values One-Hot encoded
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Pipeline: Preprocessor (One-Hot) -> Random Forest Classifier
rfModel = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=0, max_depth=6))
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Apply Random Over-sampling
oversampler = RandomOverSampler(sampling_strategy='minority', random_state=0)
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

# Train the model on the resampled data
rfModel.fit(X_resampled, y_resampled)

# Make predictions with probabilities
y_prob = rfModel.predict_proba(X_test)[:, 1]

# Adjust the threshold (experiment with different values)
threshold = 0.3
y_pred_adjusted = (y_prob > threshold).astype(int)

# Map the labels back to 'no' and 'yes'
label_mapping = {0: 'no', 1: 'yes'}
y_pred_adjusted_labels = np.vectorize(label_mapping.get)(y_pred_adjusted)

# Evaluate the adjusted predictions
accuracy = accuracy_score(y_test, y_pred_adjusted_labels)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred_adjusted_labels))

Accuracy: 0.63
              precision    recall  f1-score   support

          no       0.95      0.61      0.74      4773
         yes       0.23      0.77      0.35       723

    accuracy                           0.63      5496
   macro avg       0.59      0.69      0.55      5496
weighted avg       0.85      0.63      0.69      5496



In [ ]:

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_adjusted_labels)

# Convert confusion matrix to a DataFrame for Plotly
conf_matrix_df = pd.DataFrame(conf_matrix, index=['Actual No', 'Actual Yes'], columns=['Predicted No', 'Predicted Yes'])

# Plot heatmap using Plotly Express
fig = px.imshow(conf_matrix_df,
                labels=dict(x="Predicted", y="Actual", color="Count"),
                x=['Predicted No', 'Predicted Yes'],
                y=['Actual No', 'Actual Yes'],
                color_continuous_scale="Viridis")  # Display the actual counts in each box

# Show the plot
fig.update_layout(title_text='Heatmap of Loan')
fig.show()


In [ ]:
import plotly.express as px



df_bar = pd.DataFrame({'month': X_test['month'], 'prediction': y_pred_adjusted_labels})


fig = px.bar(df_bar, x='month', color='prediction', title='Bar Chart of Yes and No by Month',
             labels={'prediction': 'Prediction'})

#fig.update_layout({'plot_bgcolor': 'white'})

fig.show()


# Bank Holdout MINI Computation


In [ ]:
# Load the data and set the new X
bank_holdout_MINI = pd.read_csv("https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank_holdout_test_mini.csv")
X_new = bank_holdout_MINI[['age','job','marital','education','default','housing','loan','month','day_of_week', 'emp.var.rate', 'cons.price.idx', 'euribor3m', 'nr.employed']]

# Make predictions with the new data
y_new_pred = rfModel.predict(X_new)

# Make a new column in bank_holdout_MINI for the predictions
bank_holdout_MINI['y_pred'] = y_new_pred
print(bank_holdout_MINI['y_pred'])

bank_holdout_MINI['y_pred'].to_csv('team5-module2-predictions.csv', index=False)

0       no
1       no
2      yes
3       no
4       no
      ... 
405     no
406     no
407     no
408     no
409     no
Name: y_pred, Length: 410, dtype: object


In [ ]:
df = pd.DataFrame()
df["predictions"] = y_new_pred

def replaceYesNo(x):
	if(x == "yes"):
		return 1
	else:
		return 0

df['predictions'] = df['predictions'].apply(replaceYesNo)

df.to_csv("team5-module2-predictions.csv", index=False)

# Visualizations
